In [1]:
import json
from pathlib import Path

import pandas as pd
pd.set_option("display.max_colwidth", None)
import camelot
from pve_parser import process_tables

In [2]:
path = Path('pve.json')
conf = json.loads(path.read_text(encoding='utf8'))
meta = conf['meta']
patterns = conf['regex']

In [3]:
def read_tables(path, pages):
    return camelot.read_pdf(path, pages=pages, strip_text='\n')

In [6]:
pages = conf['bestandsbeschrijvingen']['OBO']['paginas']
path = 'programma-van-eisen-hoger-onderwijs.pdf'
tables = read_tables(path, pages)
dfs = [table.df for table in tables]
new_dfs = process_tables(dfs, patterns)

In [7]:
new_dfs[5]

,omschrijving,formaat_lengte,verplicht,definitie_toelichting,alternatieve_naam,dtype
0,Recordsoort,AN3..3,True,Aanduiding van het record dat geleverd wordt. Waarde “ISG”.,recordsoort,string
1,Burgerservicenummer,AN9..9,False,Het burgerservicenummer (BSN) is het nummer dat de natuurlijke persoon uniek identificeert in overheidsadministraties. Indien nodig voorzien van voorloopnul.,burgerservicenummer,string
2,Onderwijsnummer,AN9..9,False,Het uniek identificerende nummer dat aan een onderwijsontvanger wordt toegekend indien deze niet of niet verifieerbaar over een BSN beschikt. Indien nodig voorzien van voorloopnul.,onderwijsnummer,string
3,Inschrijvingvolgnummer,AN20,True,"Een door de instelling aan de inschrijving toegekend volgnummer ten behoeve van de registratie van een inschrijving bij DUO. Het inschrijvingvolgnummer moet per BRIN uniek zijn. Toelichting: Het inschrijvingvolgnummer bestaat uit cijfers, hoofdletters (A-Z), kleine letters (a-z) of een combinatie daarvan.",volgnummer,string
4,Inschrijvingsvorm,AN1,True,Inschrijvingsvorm is een indicatie die aangeeft in welke vorm een student een opleiding wil volgen. Mogelijke waarden: E = Extraneus S = Student,inschrijvingsvorm,string
5,Datum inschrijving,AN8..8 (jjjjmmdd),True,De datum vanaf wanneer de betrokkene is ingeschreven (jjjjmmdd),datum_inschrijving,datum
6,Datum uitschrijving,AN8..8 (jjjjmmdd),False,De datum waarop de betrokkene is uitgeschreven Aanvullende opmerking: De laatste datum waarop de inschrijving daadwerkelijk actief was (jjjjmmdd).,datum_uitschrijving,datum
7,Eerste inschrijving,Boolean (J/N),True,"Eerste inschrijving geeft aan of het om de opleiding van eerste inschrijving van de student gaat, te weten: 1° opleiding waarvoor een student het collegegeld, bedoeld in de artikelen 7.43, eerste lid, van de wet, is verschuldigd en waarvoor geen vermindering of vrijstelling van het betalen van collegegeld op grond van de wet is verkregen, tenzij er sprake is van een vermindering als bedoeld in artikel 7.48, derde en vierde lid, of, 2° opleiding waarvoor een persoon die het collegegeld, bedoeld in artikelen 7.43, tweede lid of 7.44 van de wet is verschuldigd, zich als eerste heeft ingeschreven. Toelichting: Bij meerdere eerste inschrijvingen bepalen de toeleverende instellingen onderling welke inschrijving van een student een eerste inschrijving betreft. Bij studenten uit het schakelprogramma moet de eerste inschrijving altijd met waarde N aangeleverd worden.",eerste_inschrijving,boolean
8,Datum eerste aanlevering,AN8..8 (jjjjmmdd),True,Datum waarop de inschrijving voor het eerst bij DUO is ontvangen (jjjjmmdd).,datum_eerste_aanlevering,datum
9,Onderwijsvorm,AN2..2,True,Uitputtende lijst ten behoeve van het Hoger Onderwijs voor de manier waarop de kennisoverdracht is ingericht wat betreft fysieke aanwezigheid op de onderwijslocatie en de duur daarvan. Mogelijke waarden: VT = Voltijd DT = Deeltijd DU = Duaal,onderwijsvorm,string


In [19]:
def table_to_json(df):
    return df.set_index('omschrijving').to_dict(orient='index')

def create_json(tables, beschrijvingen, config):
    tabelnamen = beschrijvingen['tabelnamen']
    recordsoorten = {k.lower():v for k,v in tabelnamen.items()}
    excl = ['tabelnamen']
    eigenschappen = {k:v for k,v in beschrijvingen.items() if k not in excl}
    velden = {
        k.lower():table_to_json(tables[i])
        for i,k in enumerate(tabelnamen)
    }
    data = {
        'meta': config['meta'],
        'eigenschappen': eigenschappen,
        'recordsoorten': recordsoorten,
        'velden': velden,
    }
    return json.dumps(data, indent=4)

In [21]:
bestand = 'OBO'
beschr = conf['bestandsbeschrijvingen'][bestand]
data = create_json(new_dfs, beschr, conf)
path = Path(f'tabledefs.{bestand.lower()}.json')
path.write_text(data, encoding='utf8')

25983